# Bulilding a fully connected neural network

## Imports

In [1]:

import torch
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets # Has standard datasets we can import in a nice and easy way
import torchvision.transforms as transforms # Transformations we can perform on our dataset

## Dataset

In [2]:
batch_size = 64 # no of samples nn sees at a time

train_dataset = datasets.MNIST(
    root = 'mnist_dataset/',
    train = True, #train is set to true to download the train dataset
    transform = transforms.ToTensor(),
    download = True,
) # this is used to download the mnist dataset which is available in torchvision and the transform is used to convert it to a tensor here


train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True) # The DataLoader() returns an iterator which will generate batches of the selected batch_size as tuples of (data, labels)

test_dataset = datasets.MNIST(
    root = 'mnist_dataset/',
    train = False,
    transform = transforms.ToTensor(),
    download = True,
)

test_loader = DataLoader(dataset = test_dataset, batch_size= batch_size, shuffle = True)
#similar to the train dataset the same ideas are used here


## Building the model

### CNN

In [3]:
#TODO: Read up on Convolutional layers a bit more
class CNN(nn.Module):
    def __init__(self, in_channel = 1, num_classes = 10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels=8, kernel_size = (3, 3), stride = (1,1), padding = (1,1)) 
        # This layer applies convolution operations to the input image, learning features and patterns. 
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride = (2,2))
        # pooling layers reduce the spatial dimensions of the feature maps while retaining the most important information. 
        self.conv2 = nn.Conv2d(in_channels=8, out_channels = 16, kernel_size=(3,3), stride =(1,1), padding = (1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x



In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
in_channels = 1
num_classes = 10
learning_rate = 0.001
num_epochs = 5

In [5]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [6]:
for epoch in range(num_epochs):
    print(f'Epoch : {epoch}')
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)

        #forward propogation

        scores = model(data)
        loss = criterion(scores,targets) #crossentropy loss applies softmax function so no need to apply to the output

        #zeros the previous grads to not have conflicts
        optimizer.zero_grad()

        #back_propogation
        loss.backward() #calculates gradient

        #gradient descent or adam step
        optimizer.step() #adjusts parameters to minimize loss

        

Epoch : 0
Epoch : 1
Epoch : 2
Epoch : 3
Epoch : 4


In [9]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad(): #to avoid gradient calculations as we don't need to back propogate
        for x, y in loader:
            x = x.to(device = device)
            y = y.to(device = device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy"
            f" {float(num_correct)/float(num_samples)*100:.2f}"
        )
    model.train()


In [10]:
#CNN 
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Got 59195 / 60000 with accuracy 98.66
Got 9855 / 10000 with accuracy 98.55
